# Import statements
Following packages are necessary:

In [1]:
import sys
sys.path.append("D:\Julian\workspace\endolas\packages")

import random
from endolas import utils
from endolas import ccall

import tensorflow as tf
import numpy as np
import segmentation_models as sm

from endolas import UNet
from endolas import preprocess_input as pre_une
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score
from endolas import LASTENSequence
from matplotlib import pyplot as plt

Segmentation Models: using `tf.keras` framework.


# Checks
The version of tensorflow as well as the GPU support are checked.

In [2]:
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.list_physical_devices('GPU')

2.2.0-rc3


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  # Disable all GPUS
  tf.config.set_visible_devices([], 'GPU')
  visible_devices = tf.config.get_visible_devices()
  for device in visible_devices:
    assert device.device_type != 'GPU'
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

print(tf.config.get_visible_devices('GPU'))
print(tf.config.get_visible_devices('CPU'))

[]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


# 3.3) Supervised Segmentation without augmentation
A U-Net is trained here to perform a semantic segmentation of laser points in the LASTEN2 data set.

# Data
Import training images

In [4]:
tf.keras.backend.clear_session()

#path_train = r'D:\Julian\data\LASTEN2\train'
#path_validation = r'D:\Julian\data\LASTEN2\validation'
#path_test = r'D:\Julian\data\LASTEN2\test'

path_train = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2/train'
path_validation = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2/validation'
path_test = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2/test'

width = 384 #768
height = 384 #768

# Seeding
Seeds are set to ensure reproducible training.

In [5]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Network
A U-Net based network is instantiated with keras to run a semantic segmentation.

In [6]:
model = UNet(filters=32, input_shape=(height, width, 1))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 384, 384, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 384, 384, 32) 288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 384, 384, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 384, 384, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

# Data Preparation
The generator sequence for training is built.

In [7]:
train_gen = LASTENSequence(path_train,
                           batch_size=1,
                           preprocess_input=pre_une,
                           shuffle=True,
                           augment=False,
                           width=width,
                           height=height)

validation_gen = LASTENSequence(path_validation,
                                batch_size=1,
                                preprocess_input=pre_une,
                                width=width,
                                height=height)

#test_gen = LASTENSequence(path_train,
#                          batch_size=2,
#                          preprocess_input=pre_une,
#                          width=width,
#                          height=height)

# Training Preparation
Prepare the settings for training the model.

In [8]:
aim_path= "3_3_segmentation"

save_path = "../results/" + aim_path

logger = CSVLogger(save_path + "/log")
checker = ModelCheckpoint(save_path + "/weights.{epoch:02d}.hdf5", period=10)
checker_best = ModelCheckpoint(save_path + "/best_weights.hdf5", save_best_only=True)
timelogger = ccall.TimeHistory(save_path)
#tensorboard_callback = tf.keras.callbacks.TensorBoard("../tf_logs/"+aim_path, histogram_freq=1)


callbacks = [logger, checker, checker_best, timelogger]#, tensorboard_callback]

model.compile(optimizer='adam',
              loss = dice_loss,
              metrics = ['accuracy', iou_score])

# Training
The model is fitted with a generator.

In [9]:
model.fit(train_gen,
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=validation_gen,
                    validation_freq=1,
                    #use_multiprocessing=True,
                    workers=10,
                    max_queue_size=32)

Epoch 1/100
160/160 [==============================] - 324s 2s/step - loss: 0.6722 - accuracy: 0.9279 - iou_score: 0.2043 - val_loss: 0.9007 - val_accuracy: 0.9544 - val_iou_score: 0.0528
Epoch 2/100
160/160 [==============================] - 315s 2s/step - loss: 0.3620 - accuracy: 0.9608 - iou_score: 0.4701 - val_loss: 0.3770 - val_accuracy: 0.9637 - val_iou_score: 0.4526
Epoch 3/100
160/160 [==============================] - 315s 2s/step - loss: 0.2667 - accuracy: 0.9642 - iou_score: 0.5793 - val_loss: 0.2594 - val_accuracy: 0.9644 - val_iou_score: 0.5881
Epoch 4/100
160/160 [==============================] - 315s 2s/step - loss: 0.2556 - accuracy: 0.9643 - iou_score: 0.5933 - val_loss: 0.2479 - val_accuracy: 0.9644 - val_iou_score: 0.6028
Epoch 5/100
160/160 [==============================] - 315s 2s/step - loss: 0.2485 - accuracy: 0.9643 - iou_score: 0.6023 - val_loss: 0.2424 - val_accuracy: 0.9645 - val_iou_score: 0.6100
Epoch 6/100
160/160 [==============================] - 319s 

160/160 [==============================] - 316s 2s/step - loss: 0.2154 - accuracy: 0.9647 - iou_score: 0.6458 - val_loss: 0.2385 - val_accuracy: 0.9643 - val_iou_score: 0.6151
Epoch 45/100
160/160 [==============================] - 315s 2s/step - loss: 0.2147 - accuracy: 0.9647 - iou_score: 0.6468 - val_loss: 0.2381 - val_accuracy: 0.9643 - val_iou_score: 0.6156
Epoch 46/100
160/160 [==============================] - 315s 2s/step - loss: 0.2137 - accuracy: 0.9647 - iou_score: 0.6480 - val_loss: 0.2367 - val_accuracy: 0.9644 - val_iou_score: 0.6175
Epoch 47/100
160/160 [==============================] - 315s 2s/step - loss: 0.2130 - accuracy: 0.9647 - iou_score: 0.6490 - val_loss: 0.2417 - val_accuracy: 0.9643 - val_iou_score: 0.6109
Epoch 48/100
160/160 [==============================] - 316s 2s/step - loss: 0.2122 - accuracy: 0.9647 - iou_score: 0.6501 - val_loss: 0.2420 - val_accuracy: 0.9643 - val_iou_score: 0.6104
Epoch 49/100
160/160 [==============================] - 315s 2s/step

160/160 [==============================] - 314s 2s/step - loss: 0.1893 - accuracy: 0.9648 - iou_score: 0.6817 - val_loss: 0.2459 - val_accuracy: 0.9642 - val_iou_score: 0.6055
Epoch 88/100
160/160 [==============================] - 315s 2s/step - loss: 0.1890 - accuracy: 0.9648 - iou_score: 0.6822 - val_loss: 0.2465 - val_accuracy: 0.9642 - val_iou_score: 0.6048
Epoch 89/100
160/160 [==============================] - 315s 2s/step - loss: 0.1885 - accuracy: 0.9648 - iou_score: 0.6829 - val_loss: 0.2476 - val_accuracy: 0.9642 - val_iou_score: 0.6034
Epoch 90/100
160/160 [==============================] - 317s 2s/step - loss: 0.1881 - accuracy: 0.9648 - iou_score: 0.6834 - val_loss: 0.2478 - val_accuracy: 0.9642 - val_iou_score: 0.6031
Epoch 91/100
160/160 [==============================] - 317s 2s/step - loss: 0.1883 - accuracy: 0.9648 - iou_score: 0.6832 - val_loss: 0.2492 - val_accuracy: 0.9642 - val_iou_score: 0.6014
Epoch 92/100
160/160 [==============================] - 315s 2s/step

# Evaluation

In [10]:
X, y = validation_gen[0]
y_pred = model.predict(X)

plt.imsave(save_path + "/y.png",y[0,:,:,0], cmap="gray")
plt.imsave(save_path + "/y_pred.png",y_pred[0,:,:,0], cmap="gray")

In [11]:
test_gen = LASTENSequence(path_train,
                          batch_size=1,
                          preprocess_input=pre_une,
                          width=width,
                          height=height)